In [1]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

C:\Users\jains\anaconda3\envs\DL\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
num_epochs = 80
batch_size = 100
learning_rate = 0.001

In [4]:
transform = transforms.Compose([
    transforms.Pad(4),
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32),
    transforms.ToTensor()
])

In [5]:
train_dataset = torchvision.datasets.CIFAR10(root='./data/CIFAR10',
                                            train=True,
                                            download=True,
                                            transform=transform)

test_dataset = torchvision.datasets.CIFAR10(root='./data/CIFAR10',
                                           train=False,
                                           transform=transforms.ToTensor())

Files already downloaded and verified


In [6]:
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                          batch_size=batch_size,
                                          shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                         batch_size=batch_size,
                                         shuffle=False)

In [7]:
def conv3x3(in_channels, out_channels, stride=1):
    return nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)

In [8]:
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1, downsample=None):
        super(ResidualBlock, self).__init__()
        self.conv1 = conv3x3(in_channels, out_channels, stride)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(out_channels, out_channels)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.downsample = downsample
        
    def forward(self, x):
        residual = x
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)
        if self.downsample:
            residual = self.downsample(x)
        out += residual
        out = self.relu(out)
        return out

In [9]:
class ResNet(nn.Module):
    def __init__(self, block, layers, num_classes=10):
        super(ResNet, self).__init__()
        self.in_channels = 16
        self.conv = conv3x3(3, 16)
        self.bn = nn.BatchNorm2d(16)
        self.relu = nn.ReLU(inplace=True)
        self.layer1 = self.make_layer(block, 16, layers[0])
        self.layer2 = self.make_layer(block, 32, layers[1], 2)
        self.layer3 = self.make_layer(block, 64, layers[2], 2)
        self.avg_pool = nn.AvgPool2d(8)
        self.fc = nn.Linear(64, num_classes)
        
    def make_layer(self, block, out_channels, blocks, stride=1):
        downsample = None
        if (stride != 1) or (self.in_channels != out_channels):
            downsample = nn.Sequential(
                conv3x3(self.in_channels, out_channels, stride=stride),
                nn.BatchNorm2d(out_channels)
            )
        layers = []
        layers.append(block(self.in_channels, out_channels, stride, downsample))
        self.in_channels = out_channels
        for i in range(1, blocks):
            layers.append(block(out_channels, out_channels))
        return nn.Sequential(*layers)
    
    def forward(self, x):
        out = self.conv(x)
        out = self.bn(out)
        out = self.relu(out)
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.avg_pool(out)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out

In [10]:
model = ResNet(ResidualBlock, [2, 2, 2]).to(device)

In [11]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [12]:
def update_lr(optimizer, lr):
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

In [13]:
total_step = len(train_loader)
curr_lr = learning_rate
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        
        optimizer.zero_grad()
        
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print('Epoch: [{}/{}], Step: [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, i+1, total_step, loss.item()))
            
    if (epoch+1) % 20 == 0:
        curr_lr /= 3
        update_lr(optimizer, curr_lr)

Epoch: [1/80], Step: [100/500], Loss: 1.5446
Epoch: [1/80], Step: [200/500], Loss: 1.5082
Epoch: [1/80], Step: [300/500], Loss: 1.1811
Epoch: [1/80], Step: [400/500], Loss: 1.1407
Epoch: [1/80], Step: [500/500], Loss: 1.0501
Epoch: [2/80], Step: [100/500], Loss: 1.0680
Epoch: [2/80], Step: [200/500], Loss: 1.1633
Epoch: [2/80], Step: [300/500], Loss: 1.2534
Epoch: [2/80], Step: [400/500], Loss: 0.9440
Epoch: [2/80], Step: [500/500], Loss: 1.0741
Epoch: [3/80], Step: [100/500], Loss: 0.8238
Epoch: [3/80], Step: [200/500], Loss: 0.9601
Epoch: [3/80], Step: [300/500], Loss: 0.7115
Epoch: [3/80], Step: [400/500], Loss: 0.7494
Epoch: [3/80], Step: [500/500], Loss: 0.8218
Epoch: [4/80], Step: [100/500], Loss: 1.0098
Epoch: [4/80], Step: [200/500], Loss: 0.9113
Epoch: [4/80], Step: [300/500], Loss: 0.7307
Epoch: [4/80], Step: [400/500], Loss: 0.8018
Epoch: [4/80], Step: [500/500], Loss: 0.7001
Epoch: [5/80], Step: [100/500], Loss: 0.7055
Epoch: [5/80], Step: [200/500], Loss: 0.5314
Epoch: [5/

Epoch: [37/80], Step: [100/500], Loss: 0.2379
Epoch: [37/80], Step: [200/500], Loss: 0.3062
Epoch: [37/80], Step: [300/500], Loss: 0.2138
Epoch: [37/80], Step: [400/500], Loss: 0.1040
Epoch: [37/80], Step: [500/500], Loss: 0.2208
Epoch: [38/80], Step: [100/500], Loss: 0.2139
Epoch: [38/80], Step: [200/500], Loss: 0.2344
Epoch: [38/80], Step: [300/500], Loss: 0.2208
Epoch: [38/80], Step: [400/500], Loss: 0.1653
Epoch: [38/80], Step: [500/500], Loss: 0.2203
Epoch: [39/80], Step: [100/500], Loss: 0.2694
Epoch: [39/80], Step: [200/500], Loss: 0.2089
Epoch: [39/80], Step: [300/500], Loss: 0.1760
Epoch: [39/80], Step: [400/500], Loss: 0.1721
Epoch: [39/80], Step: [500/500], Loss: 0.3229
Epoch: [40/80], Step: [100/500], Loss: 0.2006
Epoch: [40/80], Step: [200/500], Loss: 0.2823
Epoch: [40/80], Step: [300/500], Loss: 0.1574
Epoch: [40/80], Step: [400/500], Loss: 0.2523
Epoch: [40/80], Step: [500/500], Loss: 0.3011
Epoch: [41/80], Step: [100/500], Loss: 0.2721
Epoch: [41/80], Step: [200/500], L

Epoch: [72/80], Step: [500/500], Loss: 0.1273
Epoch: [73/80], Step: [100/500], Loss: 0.0910
Epoch: [73/80], Step: [200/500], Loss: 0.1597
Epoch: [73/80], Step: [300/500], Loss: 0.1364
Epoch: [73/80], Step: [400/500], Loss: 0.0860
Epoch: [73/80], Step: [500/500], Loss: 0.1874
Epoch: [74/80], Step: [100/500], Loss: 0.1447
Epoch: [74/80], Step: [200/500], Loss: 0.1439
Epoch: [74/80], Step: [300/500], Loss: 0.0921
Epoch: [74/80], Step: [400/500], Loss: 0.0689
Epoch: [74/80], Step: [500/500], Loss: 0.1902
Epoch: [75/80], Step: [100/500], Loss: 0.1325
Epoch: [75/80], Step: [200/500], Loss: 0.1033
Epoch: [75/80], Step: [300/500], Loss: 0.0896
Epoch: [75/80], Step: [400/500], Loss: 0.1741
Epoch: [75/80], Step: [500/500], Loss: 0.1497
Epoch: [76/80], Step: [100/500], Loss: 0.0923
Epoch: [76/80], Step: [200/500], Loss: 0.1338
Epoch: [76/80], Step: [300/500], Loss: 0.0738
Epoch: [76/80], Step: [400/500], Loss: 0.1662
Epoch: [76/80], Step: [500/500], Loss: 0.0775
Epoch: [77/80], Step: [100/500], L

In [14]:
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        predicted = outputs.argmax(1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        
    print('Accuracy of the model on the test images: {} %'.format(100 * correct / total))

Accuracy of the model on the test images: 88.45 %
